In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import keras


In [ ]:

from numpy import pi
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pywt

from sklearn.neighbors import NearestNeighbors
from scipy.special import gamma,psi

In [ ]:
#!pip install pyentrp
#from pyentrp import entropy as ent
#!pip download pyentrp -d ./pyentrp/
 # linear algebra
import os

!pwd

!cd /kaggle/input/packages2
!pwd
!pip install pyentrp --no-index --find-links=file:///kaggle/input/packages2/
from pyentrp import entropy as ent

In [ ]:
df_train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')


In [ ]:
files = os.listdir('/kaggle/input/hms-harmful-brain-activity-classification/train_eegs')

In [ ]:
len(files)

In [ ]:
df_train.info()

In [ ]:
df_train.head()

In [ ]:
train = pd.read_parquet('/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/1000913311.parquet')
test= pd.read_parquet('/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/3911565283.parquet')

In [ ]:
len(train),len(test)

In [ ]:
len(df_train),len(df_train['label_id'].unique())

In [ ]:
df_train['expert_consensus'].unique()

In [ ]:
print("length of dataframe --- ",len(df_train))
print("length of expert_consensus --- ",df_train['expert_consensus'].count())

df_train.expert_consensus.value_counts().plot(kind = 'bar')
### note each class do not have exactly equal no of cases ,
### so we will try to filter out bias by taking equal amount of sample in every class

In [ ]:
 df_train.groupby(['expert_consensus']).size()

In [ ]:
df_train


In [ ]:
## no of parquet files associated with patient id
x = df_train['patient_id'].value_counts()


In [ ]:
# trying to find no of samples under each ccategory for every  patient 
df_train.groupby(['patient_id', 'expert_consensus']).size()

In [ ]:
samp_df = df_train.groupby(["patient_id","expert_consensus"]).size()
samp_df.to_excel("patient_expert_consensus.xlsx")

In [ ]:

def s_entropy(freq_list):
    ''' This function computes the shannon entropy of a given frequency distribution.
    USAGE: shannon_entropy(freq_list)
    ARGS: freq_list = Numeric vector representing the frequency distribution
    OUTPUT: A numeric value representing shannon's entropy'''
    freq_list = [element for element in freq_list if element != 0]
    sh_entropy = 0.0
    for freq in freq_list:
        sh_entropy += freq * np.log(freq)
    sh_entropy = -sh_entropy
    return(sh_entropy)

def p_entropy(op):
    ordinal_pat = op
    max_entropy = np.log(len(ordinal_pat))
    p = np.divide(np.array(ordinal_pat), float(sum(ordinal_pat)))
    return(s_entropy(p)/max_entropy)


In [ ]:
def renyi_entropy(data, alpha):
    
    _, ele_counts = np.unique( np.around(data, decimals = 0 ), return_counts = True )
    summation = np.sum( np.log( ( ele_counts / len(data)) ** alpha ) ) 
    reny_ent = (1 / (1 - alpha)) * summation
    return np.array(reny_ent)

def permutation_ent(data, order=2):
    if order < 2:
        order = 2
    
    data = np.around(data, decimals = 0)
    perm_ent = ent.permutation_entropy(data, order = order, normalize = True)
    return np.array(perm_ent)

def tsallis_ent(data, alpha):
    _, ele_counts = np.unique( np.around(data, decimals = 0 ), return_counts = True )
    summation = np.sum( np.log( ( ele_counts / len(data)) ** alpha ) ) 
    tsa_ent =  (1 / (alpha - 1)) * ( 1 - summation) 
    return np.array(tsa_ent)

def kraskov_ent(data, k):
    #if k < 1:
        #k = 1
    
    #k=int(k)
    knn = NearestNeighbors(n_neighbors=k)
    X = np.around(data, decimals = 0 ).reshape(-1,1)
    knn.fit(X)
    r, _ = knn.kneighbors(X)
    n, d = X.shape
    volume_unit_ball = (pi**(.5*d)) / gamma(.5*d + 1)    
    kra_ent = (d*np.mean(np.log(r[:,-1] + np.finfo(X.dtype).eps))+ np.log(volume_unit_ball)
               + psi(n) - psi(k))
    return np.array(kra_ent)

In [ ]:
def DWT(data,wavelet_family = 'db10', level = 4):

    #data = np.load("../Healthcare_signal_processing/Datasets/Bonn/data_all.npz")
    db = pywt.Wavelet(wavelet_family)
  
    cA4, cD4, cD3, cD2, cD1 = pywt.wavedec(data, db, level = level)
    #print(samp.shape,len(cA4),len(cD4),len(d3),len(d2),len(d1))

    return [cA4, cD4, cD3, cD2, cD1]

In [ ]:
import matplotlib.pyplot as plt
groups = [0, 1, 2, 3,4]
columns=['A - A4','A - D4','A - D3','A - D2','A - D1']
X_train = pd.read_parquet('/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/1000913311.parquet')
for sig in X_train.columns:
    sig_DWT = DWT(X_train[sig])
    i = 1 
    plt.figure(figsize=(20,10))
    plt.title(sig,loc='center')
    for group in groups:
        plt.subplot(len(groups), 1, i)
        
        plt.plot(sig_DWT[group])
        plt.title(sig+' -- '+columns[group], y=0.5, loc='right')
        i += 1

    plt.show()

In [ ]:
def extract_enctropy_features (signal,re_alpha =2 ,pe_order=2,tsa_order=3,kras_order=3):
    decomp_wavelet = DWT(signal) ## DWT will decompose the signal into 5 coefficients 
    features=[]
    for coeff in decomp_wavelet:
        features.append(renyi_entropy(coeff,re_alpha))
        features.append(permutation_ent(coeff,pe_order))
        features.append(tsallis_ent(coeff,tsa_order))
        #features.append(kraskov_ent(coeff,kras_order))
    ## after finiding enrtropies for each coefficeint  we get 20 features - 
    return features

In [ ]:
## considering equal number of smaples in each case == 14000
"""
expert_consensus
GPD        16702
GRDA       18861
LPD        14856
LRDA       16640
Other      18808
Seizure    20933
"""
classes = ['GPD','GRDA','LPD','LPDA','Other','Seizure']
input_X = []
target_values = []
df_train_GPD = df_train[df_train['expert_consensus'] == 'GPD']
df_train_GRDA = df_train[df_train['expert_consensus'] == 'GRDA']
df_train_LPD = df_train[df_train['expert_consensus'] == 'LPD']
df_train_LRDA = df_train[df_train['expert_consensus'] == 'LRDA']
df_train_Other = df_train[df_train['expert_consensus'] == 'Other']
df_train_Seizure = df_train[df_train['expert_consensus'] == 'Seizure']

In [ ]:
df_train_GPD

In [ ]:
eeg_ids= df_train['eeg_id'].unique()
d = {}
for eeg_id in eeg_ids:
    lab = df_train[df_train['eeg_id'] == eeg_id]['expert_consensus'].values
    if all(lab):
        if lab[0] not in d.keys():
            d[lab[0]] = [eeg_id]
        else:
            d[lab[0]].append(eeg_id)
for  key,val in d.items():
    print('class ->',key ,'no of eegs file which are completely mapped to a single label',str(len(val)))

In [ ]:

'''
fin_labels = labels[np.random.choice(len(labels), size=900, replace=False)]
for label_id in fin_labels:
    temp=[]
    target_values.append(df_train_gpd[df_train_gpd['eeg_id'] == label_id][['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']].values)
    #sig_df= pd.read_parquet("/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{label_id}.parquet".format(label_id = label_id))
    for col in sig_df.columns:
        temp.extend(extract_enctropy_features(sig_df[col].values))
print(temp)
'''


In [ ]:
input_eeg = []
vote = []


In [ ]:
sample = df_train_Seizure.sort_values('seizure_vote', ascending=False).reset_index()
i, count=0, 0
patient_li=[]
while count<100:
    pat_id = sample['patient_id'][i]
    if pat_id in patient_li:
        i+=1
        continue
    patient_li.append(pat_id)
    eeg_id = sample['eeg_id'][i]
    offset_idx = int(sample['eeg_label_offset_seconds'][i]*100)
    eeg = pd.read_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')
    eeg = eeg[offset_idx:offset_idx+5000]
    temp = []
    for col in eeg.columns:
        if all(np.isnan(eeg[col])) :
            print(eeg.eeg_id.unique())
        temp.extend(extract_enctropy_features(eeg[col].values))
    input_eeg.append(temp)
    vote.append(sample[['seizure_vote' ,'lpd_vote' ,'gpd_vote', 'lrda_vote', 'grda_vote' ,'other_vote']].iloc[i].values)

    count+=1
    i+=1

In [ ]:
len(vote),len(input_eeg)

In [ ]:
len(input_eeg[0])

In [ ]:
sample = df_train_GPD.sort_values('gpd_vote', ascending=False).reset_index()
i, count=0, 0
patient_li=[]
while count<100:
    pat_id = sample['patient_id'][i]
    if pat_id in patient_li:
        i+=1
        continue
    patient_li.append(pat_id)
    eeg_id = sample['eeg_id'][i]
    offset_idx = int(sample['eeg_label_offset_seconds'][i]*100)
    eeg = pd.read_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')
    eeg = eeg[offset_idx:offset_idx+5000]
    temp = []
    for col in eeg.columns:
        if all(np.isnan(eeg[col])) :
            print(eeg.eeg_id.unique())
        temp.extend(extract_enctropy_features(eeg[col].values))
    input_eeg.append(temp)
    vote.append(sample[['seizure_vote' ,'lpd_vote' ,'gpd_vote', 'lrda_vote', 'grda_vote' ,'other_vote']].iloc[i].values)
    
    count+=1
    i+=1

In [ ]:
len(vote)

In [ ]:
sample = df_train_GRDA.sort_values('grda_vote', ascending=False).reset_index()
i, count=0, 0
patient_li=[]
while count<100:
    pat_id = sample['patient_id'][i]
    if pat_id in patient_li:
        i+=1
        continue
    patient_li.append(pat_id)
    eeg_id = sample['eeg_id'][i]
    offset_idx = int(sample['eeg_label_offset_seconds'][i]*100)
    eeg = pd.read_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')
    eeg = eeg[offset_idx:offset_idx+5000]
    temp = []
    for col in eeg.columns:
        if all(np.isnan(eeg[col])) :
            print(eeg.eeg_id.unique())
        temp.extend(extract_enctropy_features(eeg[col].values))
    input_eeg.append(temp)
    vote.append(sample[['seizure_vote' ,'lpd_vote' ,'gpd_vote', 'lrda_vote', 'grda_vote' ,'other_vote']].iloc[i].values)
    
    count+=1
    i+=1

In [ ]:
len(vote)

In [ ]:
sample = df_train_LPD.sort_values('lpd_vote', ascending=False).reset_index()
i, count=0, 0
patient_li=[]
while count<100:
    pat_id = sample['patient_id'][i]
    if pat_id in patient_li:
        i+=1
        continue
    patient_li.append(pat_id)
    eeg_id = sample['eeg_id'][i]
    offset_idx = int(sample['eeg_label_offset_seconds'][i]*100)
    eeg = pd.read_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')
    eeg = eeg[offset_idx:offset_idx+5000]
    temp = []
    for col in eeg.columns:
        if all(np.isnan(eeg[col])) :
            print(eeg.eeg_id.unique())
        temp.extend(extract_enctropy_features(eeg[col].values))
    input_eeg.append(temp)
    vote.append(sample[['seizure_vote' ,'lpd_vote' ,'gpd_vote', 'lrda_vote', 'grda_vote' ,'other_vote']].iloc[i].values)
    
    count+=1
    i+=1

In [ ]:
len(vote)

In [ ]:
sample = df_train_LRDA.sort_values('lrda_vote', ascending=False).reset_index()
i, count=0, 0
patient_li=[]
while count<100:
    pat_id = sample['patient_id'][i]
    if pat_id in patient_li:
        i+=1
        continue
    patient_li.append(pat_id)
    eeg_id = sample['eeg_id'][i]
    offset_idx = int(sample['eeg_label_offset_seconds'][i]*100)
    eeg = pd.read_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')
    eeg = eeg[offset_idx:offset_idx+5000]
    temp = []
    for col in eeg.columns:
        if all(np.isnan(eeg[col])) :
            print(eeg.eeg_id.unique())
        temp.extend(extract_enctropy_features(eeg[col].values))
    input_eeg.append(temp)
    vote.append(sample[['seizure_vote' ,'lpd_vote' ,'gpd_vote', 'lrda_vote', 'grda_vote' ,'other_vote']].iloc[i].values)

    count+=1
    i+=1

In [ ]:
len(vote)

In [ ]:
sample = df_train_Other.sort_values('other_vote', ascending=False).reset_index()
i, count=0, 0
patient_li=[]
while count<100:
    pat_id = sample['patient_id'][i]
    if pat_id in patient_li:
        i+=1
        continue
    patient_li.append(pat_id)
    eeg_id = sample['eeg_id'][i]
    offset_idx = int(sample['eeg_label_offset_seconds'][i]*100)
    eeg = pd.read_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')
    eeg = eeg[offset_idx:offset_idx+5000]
    temp = []
    for col in eeg.columns:
        if all(np.isnan(eeg[col])) :
            print(eeg.eeg_id.unique())
        temp.extend(extract_enctropy_features(eeg[col].values))
    input_eeg.append(temp)
    vote.append(sample[['seizure_vote' ,'lpd_vote' ,'gpd_vote', 'lrda_vote', 'grda_vote' ,'other_vote']].iloc[i].values)
    

    count+=1
    i+=1

In [ ]:
len(vote)

In [ ]:
###TEST data 
df_test = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/test.csv")
df_test
eeg_test = pd.read_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/3911565283.parquet')
eeg_test
temp_test = []

for col in eeg_test.columns:
    if all(np.isnan(eeg_test[col])) :
        print(eeg_test.eeg_id.unique())
    temp_test.extend(extract_enctropy_features(eeg_test[col].values))  

In [ ]:
input_eeg_test= np.array(temp_test)

In [ ]:
input_eeg_test = input_eeg_test.reshape(1,300)

In [ ]:
len(vote),len(input_eeg)

In [ ]:
vote = np.array(vote)
input_eeg = np.array(input_eeg)

In [ ]:
vote[0:100]

In [ ]:
vote[101:200]

In [ ]:
vote[201:300]

In [ ]:
vote[301:400]

In [ ]:
vote[401:500]

In [ ]:
vote[501:600]

In [ ]:
vote = np.array(vote)
max = vote.max(axis=1).reshape(-1, 1)

# Setting max values as 1 and other as 0
final_vote = np.where(vote == max, 1, 0)

In [ ]:
final_vote[400:500]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
input_eeg=scaler.fit_transform(input_eeg)
input_eeg_test = scaler.fit_transform(input_eeg_test)

## the features are extracted and further can be used to train a model


In [ ]:
def ANN(X,Y,input_eeg_test):
    acc = []
    models = []
    predictions = []
    #ten_fold_score = cross_val_score(classify, X, Y, cv = 10)
    for _ in range(2):
        model = Sequential()
        model.add(Dense(200, input_dim=X.shape[1], activation='relu'))
        #model.add(Dense(300,input_dim=X.shape[1], activation='relu'))
        model.add(Dense(150, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(6, activation='softmax'))
        # Compile model
        mse = keras.losses.MeanSquaredError(reduction="sum_over_batch_size", name="mean_squared_error")
        model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
        # fit the keras model on the dataset

        X_train , X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15)
        model.fit(np.array(X_train), np.array(Y_train), epochs=50, batch_size=5, verbose=1)
        
        
        scores = model.evaluate(np.array(X_test),Y_test)
        for i,m in enumerate(model.metrics_names):
            print(str(m) +'--'+str(scores[i]))
        models.append(model)
        Y_pred = model.predict(np.array(np.array(input_eeg_test)))
        predictions.append(Y_pred)
    np.save("ANN", acc)
    return models,predictions

In [ ]:
final_vote.shape,input_eeg.shape


In [ ]:
input_eeg

In [ ]:
 models,preds = ANN(input_eeg,vote,input_eeg_test)

In [ ]:
preds

In [ ]:
sub_df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv')

In [ ]:
sub_df

In [ ]:
samp = preds

In [ ]:
preds

In [ ]:
sum(preds[0])

In [ ]:
output_file = 'output{}.csv'
colmns = list(sub_df.columns)
colmns.remove('eeg_id')
print(colmns)
for i in range(len(preds)):
    pred = preds[i]
    for j in range(len(colmns)):
        print(pred.shape,j)
        sub_df[colmns[j]][0] = pred[0][j]
        
    sub_df.to_csv('submission.csv',index = False)
    